# DeskDrop datasets EDA
# Part 1 - Introduction
## 1.1 Load Library

In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import collections
import numpy as np
import pandas as pd
import seaborn as sns


import geopandas, fiona
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry import Point
from functools import partial

import matplotlib.colors as colors
import matplotlib.cm as cmx

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (15, 7.5)

from datetime import datetime
import re


## 1.2 Load Data

In [91]:
articles = pd.read_csv('DS5230/shared_articles.csv')
interactions_df = pd.read_csv('DS5230/users_interactions.csv')

In [132]:
articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


## 1.3 Join Two Tables

In [222]:
df = pd.merge(articles, interactions_df, how='right', left_on='contentId', right_on='contentId')
# df = interactions_df.join(articles, left_on='contentId', left_on='contentId')


## 1.4 Missing values

In [223]:
df.isnull().sum()

timestamp_x            0
eventType_x            0
contentId              0
authorPersonId         0
authorSessionId        0
authorUserAgent    57426
authorRegion       57426
authorCountry      57426
contentType            0
url                    0
title                  0
text                   0
lang                   0
timestamp_y            0
eventType_y            0
personId               0
sessionId              0
userAgent          15496
userRegion         15507
userCountry        15496
dtype: int64

### 1.5 Number of records

In [224]:
tot = len(df.index)
print('Num of records: \t%d' % tot)

Num of records: 	72680


### 1.6 Drop columns authorUserAgent, authorRegion and authorCountry 

In [225]:
columns = ['authorUserAgent', 'authorRegion', 'authorCountry']
df.drop(columns, inplace=True, axis=1)

### 1.7 Number of records after drop NA

In [226]:
df = df.dropna(how='any')

tot = len(df.index)
print('Num of records after drop NA: \t%d' % tot)

Num of records after drop NA: 	57173


### 1.8 Rename columns 

In [227]:
df = df.rename(index=str, columns={"timestamp_x": "timestamp_interaction"})
df = df.rename(index=str, columns={"timestamp_y": "timestamp_article"})

## PART 2 Explore
### 2.1 First and last intersection happen time

In [228]:
def to_datetime(timestamp):
    return datetime.fromtimestamp(timestamp)

def to_datetime_str(timestamp):
    return to_datetime(timestamp).strftime('%Y-%m-%d %H:%M:%S')

print('First interaction: \t%s' % to_datetime_str(interactions_df['timestamp'].min()))
print('Last interaction: \t%s' % to_datetime_str(interactions_df['timestamp'].max()))

First interaction: 	2016-03-14 09:54:36
Last interaction: 	2017-02-28 14:21:51


### 2.1 userAgent Count

In [229]:
nUA = len(df['userAgent'].value_counts())
android_count = len(df[df['userAgent'] == 'Android - Native Mobile App'])
ios_count = len(df[df['userAgent'] == 'iOS - Native Mobile App'])

print('Num of distinct userAgent: \t%d' % nUA)
print('Num of interactions (Android native app): %d' % android_count)
print('Num of interactions (iOS native app): %d' % ios_count)

Num of distinct userAgent: 	1090
Num of interactions (Android native app): 6790
Num of interactions (iOS native app): 103


### 2.2 userRegion Count

In [230]:
nUR = len(df['userRegion'].value_counts())
print('Num of distinct userRegion: \t%d' % nUR)

Num of distinct userRegion: 	71


### userRegion has awkward value such as ?, 41, 33, R ...

### 2.3 userCountry Count

In [231]:
nUC = len(df['userCountry'].value_counts())
print('Num of distinct userCountry: \t%d' % nUC)

Num of distinct userCountry: 	22


In [232]:
df['userCountry'].value_counts().head()

BR    51640
US     4677
KR      239
CA      226
JP      145
Name: userCountry, dtype: int64

### 2.4 eventType Count

In [233]:
nEY = len(df['eventType_y'].value_counts())
print('Num of Distinct eventType_y: \t%d' % nEY)

Num of Distinct eventType_y: 	5


In [234]:
nEX = len(df['eventType_x'].value_counts())
print('Num of istinct eventType_x: \t%d' % nEX)

Num of istinct eventType_x: 	2


In [235]:
df['eventType_y'].value_counts()

VIEW               54697
LIKE                1256
BOOKMARK             479
FOLLOW               403
COMMENT CREATED      338
Name: eventType_y, dtype: int64

In [236]:
df['eventType_x'].value_counts()

CONTENT SHARED     56866
CONTENT REMOVED      307
Name: eventType_x, dtype: int64

### * Assign event_type strength

In [237]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

df['eventStrength'] = df['eventType_y'].apply(lambda x: event_type_strength[x])

### Drop column eventType_y

In [238]:
df = df.drop(columns=['eventType_y'])

### Renmame columns eventType_x to eventType

In [239]:
df = df.rename(index=str, columns={"eventType_x": "eventType"})

## 2.5 timestamp transfer

In [240]:
df['datetime_interaction'] = df['timestamp_interaction'].apply(lambda x: to_datetime(x))
df['datetime_article'] = df['timestamp_article'].apply(lambda x: to_datetime(x))

## 2.6 Author Count

In [241]:
nAT = len(df['authorPersonId'].value_counts())
print('Num of distinct Author: \t%d' % nAT)

Num of distinct Author: 	252


In [242]:
nA = len(df['authorSessionId'].value_counts())
print('Num of distinct Author: \t%d' % nA)

Num of distinct Author: 	2001


## 2.7 Drop columns authorSessionId, contentType, url

In [243]:
columns_drop = ['authorSessionId', 'contentType', 'url']
df.drop(columns_drop, inplace=True, axis=1)

In [244]:
df.head()

,timestamp_interaction,eventType,contentId,authorPersonId,title,text,lang,timestamp_article,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,datetime_interaction,datetime_article
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1459192561,4340306774493623681,8940341205206233829,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5...,SP,BR,1.0,2016-03-28 15:19:39,2016-03-28 15:16:01
1,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1459192667,4340306774493623681,8940341205206233829,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5...,SP,BR,1.0,2016-03-28 15:19:39,2016-03-28 15:17:47
2,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1459164960,-1578287561410088674,2811329017818501696,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,SP,BR,1.0,2016-03-28 15:19:39,2016-03-28 07:36:00
3,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1459270471,2873028073541627603,-3228450896145227905,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0,2016-03-28 15:39:48,2016-03-29 12:54:31
4,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en,1459539433,8414731042150985013,4543899740167763020,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0,2016-03-28 15:42:26,2016-04-01 15:37:13


## 2.9 Language Count

In [245]:
nL = len(df['lang'].value_counts())
print('Num of distinct language: \t%d' % nL)

Num of distinct language: 	5


In [246]:
df['lang'].value_counts()

en    37021
pt    20075
la       40
ja       31
es        6
Name: lang, dtype: int64

### * Drop la, ja, es 

In [254]:
indexNames = df[ (df['lang'] == 'la') | (df['lang'] == 'ja') | (df['lang'] == 'es') ].index


df.drop(indexNames , inplace=True)

In [256]:
df['lang'].value_counts()

en    37021
pt    20075
Name: lang, dtype: int64

In [257]:
df.to_csv('df.csv')